# サポートベクターマシン分類器(Support Vector Machine (SVM) Classifier)
Model Definition: 高次元データに適した線形カーネルを使用したSVM分類器。

Training: サンプリングされたトレーニングデータのサブセットで学習。

Prediction: 検証セットで予測を行い、モデルを評価。

Evaluation: SVMモデルの性能を分析するため、分類レポートと精度指標を表示。

In [1]:
from google.colab import drive
drive.mount('/content/drive')

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os

# Define dataset filenames
train_file = "train_1_split.csv"
val_file = "val_1_split.csv"

# Construct dynamic paths based on the current working directory
current_dir = os.getcwd()
train_path = os.path.join(current_dir, "/content/drive/MyDrive/image_analytics/data/features", train_file)
val_path = os.path.join(current_dir, "/content/drive/MyDrive/image_analytics/data/features", val_file)

# Optional check if paths exist
if not os.path.isfile(train_path):
    print(f"Warning: {train_path} が見つかりません")
if not os.path.isfile(val_path):
    print(f"Warning: {val_path} が見つかりません")

# Load the training and validation datasets
train = pd.read_csv(train_path)
val = pd.read_csv(val_path)

# Separate features and labels for the training and validation sets
X_train = train.iloc[:, 2:].astype(np.float32)
y_train = train['label']
X_val = val.iloc[:, 2:].astype(np.float32)
y_val = val['label']

# Sample a subset for training
X_train_sample, _, y_train_sample, _ = train_test_split(X_train, y_train, train_size=5000, random_state=42, stratify=y_train)

# Confirm dimensions
print("Training Features Shape:", X_train.shape)
print("Training Labels Shape:", y_train.shape)
print("Validation Features Shape:", X_val.shape)
print("Validation Labels Shape:", y_val.shape)

Mounted at /content/drive
Training Features Shape: (1024933, 176)
Training Labels Shape: (1024933,)
Validation Features Shape: (256234, 176)
Validation Labels Shape: (256234,)


## SVM Model: Training, Prediction, and Evaluation

ここでは、線形カーネルを持つサポートベクターマシン（SVM）分類器を実装し、検証データでその性能を評価します。

1. **必要なライブラリのインポート**

2. **SVM分類器の定義**:
   - `svm_clf`は線形の決定境界を使用するために`kernel='linear'`で設定され、線形分離可能なデータに適しています。
   - 再現性を維持するために`random_state=42`を指定。
   - 分析に有用な確率推定を可能にするために`probability=True`を設定。

3. **モデルのトレーニング**:
   - 分類器は、特徴量とラベルの関係を学習するために、サンプリングされたトレーニングデータ（X_train_sampleとy_train_sample）で学習。

4. **予測の実行**:
   - 学習済みの分類器を使用して、検証データセット（X_val）に対し予測。これらの予測（svm_y_pred）は、検証セットの各サンプルに対する分類器の予測を提供する。

5. **モデルの性能評価**:
   - モデルの精度を計算し、これは正しい予測の割合を反映。
   - 分類レポートは、各クラスの適合率、再現率、F1スコアなどの詳細な指標を提供し、異なるクラス間でのモデルの性能を理解するのに役立ちます。



In [ ]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import joblib

# Define the SVM Classifier with default parameters
svm_clf = SVC(kernel='linear', random_state=42, probability=True)

# Train the classifier on the sampled training data
svm_clf.fit(X_train_sample, y_train_sample)

# Make predictions on the validation data
svm_y_pred = svm_clf.predict(X_val)

# Define the new path for saving the model inside the models folder
model_save_path = "/content/drive/MyDrive/image_analytics/models/svm_classifier.pkl"

# Save the trained model
joblib.dump(svm_clf, model_save_path)

print(f"Model saved to {model_save_path}.")

# Evaluate the model
print("Support Vector Machine (SVM) Classifier Accuracy:", accuracy_score(y_val, svm_y_pred))
print("\nClassification Report:\n", classification_report(y_val, svm_y_pred))


### Model Evaluation Metrics

モデルを評価するための包括的な指標のセットを計算し、表示します。

1. **精度（**ACCURACY**）**: 正しい予測の全体的な割合。
2. **適合率（**PRECISION**）（加重）**: クラスの不均衡を考慮した、全クラスの適合率の加重平均。
3. **再現率（**RECALL**）（加重）**: 全クラスの再現率の加重平均。
4. **F1スコア（**F1 SCORE**）（加重）**: 適合率と再現率のバランスを取る、全クラスのF1スコアの加重平均。


In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Make Calculate Evaluation Metrics
# Precision, Recall, F1-Score, and Accuracy
precision = precision_score(y_val, svm_y_pred, average='weighted')
recall = recall_score(y_val, svm_y_pred, average='weighted')
f1 = f1_score(y_val, svm_y_pred, average='weighted')
accuracy = accuracy_score(y_val, svm_y_pred)

# Print the metrics
print("SVM Model Evaluation Metrics:")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


### SVM Model Performance Visualizations

ここでは、SVM分類器の性能を評価するためのいくつかの可視化を行います。

1. **混同行列作成（**CONFUSION MATRIX**）**:
  - 各クラスの真の予測と誤った予測の分布を示す混同行列のヒートマップ。
  - 分類器がどこで良好な性能を示し、どこで誤分類するかを特定するのに役立ちます。

2. **各クラスの**ROC**曲線と**AUC**値**:
  - 各クラスの真陽性率と偽陽性率のトレードオフを示す**ROC**（Receiver Operating Characteristic）曲線を表示。
  - **AUC**（Area Under the Curve）値は各クラスの分離可能性を測定し、高い**AUC**値はより良いクラス区別を示します。

3. **クラス別精度（**CLASS-WISE ACCURACY**）**:
  - 各クラスの正しい予測の総予測に対する比率として計算された、クラスごとの精度を示す棒グラフ。
  - モデルが高い精度または低い精度を示すクラスを特定するのに有用です。

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, roc_curve, auc

# Make Confusion Matrix
conf_matrix = confusion_matrix(y_val, svm_y_pred)
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
plt.title("Confusion Matrix for SVM Classifier")
plt.xlabel("Predicted Labels")
plt.ylabel("True Labels")
plt.show()

# Make ROC Curve and AUC for each class (One-vs-Rest)
y_val_binary = label_binarize(y_val, classes=sorted(set(y_val)))  # Convert labels to binary format for ROC computation
svm_y_pred_proba = svm_clf.decision_function(X_val)

# Plot ROC curve for each class
plt.figure(figsize=(10, 7))
for i in range(y_val_binary.shape[1]):
    fpr, tpr, _ = roc_curve(y_val_binary[:, i], svm_y_pred_proba[:, i])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"Class {i} (AUC = {roc_auc:.2f})")

plt.plot([0, 1], [0, 1], "k--")
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("SVM Classifier ROC Curve (One-vs-Rest)")
plt.legend(loc="best")
plt.show()

# Accuracy by Class from Confusion Matrix
class_accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)
plt.figure(figsize=(12, 6))
plt.bar(range(len(class_accuracies)), class_accuracies, color="skyblue")
plt.title("Class-wise Accuracy for SVM Classifier")
plt.xlabel("Class")
plt.ylabel("Accuracy")
plt.xticks(range(len(class_accuracies)))
plt.show()
